In [266]:
# load package
from cryptory import Cryptory
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
np.set_printoptions(suppress=True)

In [10]:
# initialise object 
# pull data from start of 2017 to present day
my_cryptory = Cryptory(from_date = "2017-01-01")

# get historical bitcoin prices from coinmarketcap
my_cryptory.extract_coinmarketcap("bitcoin").head()

,date,open,high,low,close,volume,marketcap
0,2019-05-04,5769.20,5886.89,5645.47,5831.17,17567780766,103112021259
1,2019-05-03,5505.55,5865.88,5490.20,5768.29,18720780006,101986240859
2,2019-05-02,5402.42,5522.26,5394.22,5505.28,14644460907,97330112147
3,2019-05-01,5350.91,5418.00,5347.65,5402.70,13679528236,95501110091
4,2019-04-30,5247.73,5363.26,5224.19,5350.73,13878964574,94573826827


In [43]:
all_coins_df = my_cryptory.extract_bitinfocharts("btc")
# coins of interest
bitinfocoins = ["btc", "eth", "xrp", "ltc", "dash", "xmr", "doge"]
for coin in bitinfocoins[1:]:
    all_coins_df = all_coins_df.merge(my_cryptory.extract_bitinfocharts(coin), on="date", how="left")

In [44]:
all_coins_df = all_coins_df.fillna(0)
all_coins_df.set_index('date',inplace=True)
all_coins_df.insert(0,"usd_price",1)
all_coins_df = all_coins_df.reindex(index=all_coins_df.index[::-1])
all_coins_df.head()

,usd_price,btc_price,eth_price,xrp_price,ltc_price,dash_price,xmr_price,doge_price
date,,,,,,,,
2017-01-01,1,970.988,8.233,0.00651,4.389,11.356,13.532,0.000224
2017-01-02,1,1010.000,8.182,0.00640,4.539,11.593,14.671,0.000222
2017-01-03,1,1017.000,8.811,0.00632,4.525,12.383,16.125,0.000220
2017-01-04,1,1075.000,10.440,0.00642,4.585,14.748,16.807,0.000226
2017-01-05,1,1045.000,10.479,0.00650,4.404,14.815,16.713,0.000225


In [151]:
tokens = all_coins_df.columns.tolist()
tokens

['usd_price',
 'btc_price',
 'eth_price',
 'xrp_price',
 'ltc_price',
 'dash_price',
 'xmr_price',
 'doge_price']

In [46]:
all_coins_df.tail()

,usd_price,btc_price,eth_price,xrp_price,ltc_price,dash_price,xmr_price,doge_price
date,,,,,,,,
2019-05-01,1,5354.0,161.126,0.305,73.231,114.102,63.580,0.00250
2019-05-02,1,5432.0,160.495,0.304,73.269,116.763,64.995,0.00257
2019-05-03,1,5704.0,166.240,0.309,77.391,119.319,66.477,0.00266
2019-05-04,1,5750.0,164.132,0.302,77.448,120.244,67.009,0.00264
2019-05-05,1,5793.0,163.603,0.303,77.200,118.970,68.080,0.00262


In [47]:
all_coins_df.index

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2019-04-26', '2019-04-27', '2019-04-28', '2019-04-29',
               '2019-04-30', '2019-05-01', '2019-05-02', '2019-05-03',
               '2019-05-04', '2019-05-05'],
              dtype='datetime64[ns]', name='date', length=855, freq=None)

In [150]:
data = []

coinNames = iter(all_coins_df)
next(coinNames)

for index, coin in enumerate(tokens):
    coin_chart_info = go.Scatter(
        x = all_coins_df.index,
        y = all_coins_df[coin].tolist(),
        mode = 'lines',
        name = coin[:-6])
    data.append(coin_chart_info)
    tokens[index] = coin[:-6]

KeyError: 'usd'

In [96]:
layout = go.Layout(
    title = "Historic Price Over Time",
    autosize=True,
    showlegend=True,
    legend=dict(x=0.9, y=0.9))

fig = dict(data=data, layout = layout)
py.iplot(fig, filename='stacked-area-plot-hover')

In [50]:
initial_price = all_coins_df.head(1)
initial_price

,usd_price,btc_price,eth_price,xrp_price,ltc_price,dash_price,xmr_price,doge_price
date,,,,,,,,
2017-01-01,1,970.988,8.233,0.00651,4.389,11.356,13.532,0.000224


In [51]:
initial_price = np.array(initial_price)[0]
initial_price

array([  1.      , 970.988   ,   8.233   ,   0.00651 ,   4.389   ,
        11.356   ,  13.532   ,   0.000224])

In [52]:
INVEST = 1000

In [53]:
initial_weights = np.ones(len(tokens)) / len(tokens)
initial_weights

array([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125])

In [54]:
initial_allocation = INVEST * initial_weights / initial_price
initial_allocation[np.isinf(initial_allocation)] = 0
initial_allocation

array([   125.        ,      0.12873486,     15.18280092,  19201.22887865,
           28.48029164,     11.00739697,      9.23736329, 558035.71428571])

In [55]:
initial_positions = initial_allocation * initial_price
initial_positions

array([125., 125., 125., 125., 125., 125., 125., 125.])

In [56]:
initial_portfolio_value = initial_positions.sum().sum()
initial_portfolio_value

1000.0

In [57]:
tokens

['usd', 'btc', 'eth', 'xrp', 'ltc', 'dash', 'xmr', 'doge']

In [58]:
hodl = {}
rb = {}

current_allocation = initial_allocation

for i, current_price in all_coins_df.iterrows():
    # hodl positions
    hodl[i] = initial_allocation * current_price
    
    # current positions
    current_positions = current_allocation * current_price
    rb[i] = current_positions
    
    # rebalance
    current_portfolio_value = current_positions.sum()
    if(current_price.min() > 0):
        current_allocation = current_portfolio_value * initial_weights / current_price

In [59]:
hodl = pd.DataFrame(hodl).T
rb = pd.DataFrame(rb).T

In [61]:
all_coins_df.min()

usd_price       1.000000
btc_price     781.292000
eth_price       8.182000
xrp_price       0.000000
ltc_price       3.729000
dash_price      0.000000
xmr_price      10.490000
doge_price      0.000201
dtype: float64

In [73]:
hodl_value.tolist()

[1000.0,
 1018.4218283473165,
 1047.9490301218143,
 1119.4579647216772,
 1111.8803615462866,
 1036.2044863482297,
 987.070568575263,
 1014.8646766677101,
 1012.375230975967,
 1029.4759856902037,
 999.2093019636842,
 955.6433525646219,
 970.8785646399747,
 978.7234052669639,
 968.4728235462508,
 969.1685719012285,
 1005.3371040992783,
 1013.6357451147646,
 1023.1783076653343,
 1040.149566234021,
 1038.1890467433736,
 1040.6916119259658,
 1036.7297610134199,
 1020.7555911546704,
 1007.1198150172322,
 1015.8487432770288,
 1030.755293965412,
 1031.965535653161,
 1032.7231663573764,
 1040.995095752039,
 1056.178465303558,
 1057.4302830060642,
 1071.2369553055933,
 1073.7650047149139,
 1086.3786176163985,
 1081.418925968721,
 1077.781816277818,
 1087.0377897148,
 1087.286128421807,
 1070.719975477048,
 1048.8279144984508,
 1070.504504081653,
 1068.7906715492009,
 1064.4192067703414,
 1086.8880845224835,
 1124.4445185101229,
 1133.3887153545493,
 1131.609689515787,
 1142.9856649714602,
 1152.

In [67]:
hodl_value = hodl.sum(axis=1)
rb_value = rb.sum(axis=1)

hodl_perf = hodl_value.iloc[-1] / hodl_value.iloc[0]
rb_perf = rb_value.iloc[-1] / rb_value.iloc[0]

In [79]:
hodlInfo = go.Scatter(
        x = hodl_value.index,
        y = hodl_value.tolist(),
        mode = 'lines',
        name = 'hodl {:.1f}%'.format(hodl_perf * 100))

rebalanceInfo = go.Scatter(
        x = rb_value.index,
        y = rb_value.tolist(),
        mode = 'lines',
        name = 'rebalance {:.1f}%'.format(rb_perf * 100))

In [99]:
layout = go.Layout(
    title = "Portfolio value over time",
    autosize=True,
    showlegend=True,
    legend=dict(x=0.8, y=0.9))

fig = dict(data=[hodlInfo, rebalanceInfo], layout = layout)
py.iplot(fig)

In [261]:
funds = [
    {
    'coins': ['eth_price', 'btc_price', 'usd_price'],
    'ratio': [0.4, 0.4, 0.2],
    'rebalance': 7,
    'name': 'ETH40, BTC40, USD20 @ 7',
    },
    {
    'coins': ['eth_price', 'btc_price', 'usd_price'],
    'ratio': [0.25, 0.25, 0.5],
    'rebalance': 30,
    'name': 'ETH25, BTC25, USD20 @ 30',
    },
    {
    'coins': ['eth_price', 'btc_price'],
    'ratio': [0.5, 0.5],
    'rebalance': 15,
    'name': 'ETH50, BTC50 @ 15',
    },
    {
    'coins': ['eth_price', 'btc_price'],
    'ratio': [0.5, 0.5],
    'rebalance': 0,
    'name': 'ETH50, BTC50 @ HODL',
    }]

for index, fund in enumerate(funds):
    print(fund)

{'coins': ['eth_price', 'btc_price', 'usd_price'], 'ratio': [0.4, 0.4, 0.2], 'rebalance': 7, 'name': 'ETH40, BTC40, USD20 @ 7'}
{'coins': ['eth_price', 'btc_price', 'usd_price'], 'ratio': [0.25, 0.25, 0.5], 'rebalance': 30, 'name': 'ETH25, BTC25, USD20 @ 30'}
{'coins': ['eth_price', 'btc_price'], 'ratio': [0.5, 0.5], 'rebalance': 15, 'name': 'ETH50, BTC50 @ 15'}
{'coins': ['eth_price', 'btc_price'], 'ratio': [0.5, 0.5], 'rebalance': 0, 'name': 'ETH50, BTC50 @ HODL'}


In [262]:
fund_returns = {}
for index, fund in enumerate(funds):
    
    coin_prices = all_coins_df.loc[:,fund['coins']]
    
    initial_weights = np.array(fund['ratio'])
    
    initial_price = coin_prices.head(1)
    initial_price = np.array(initial_price)[0]
    
    initial_allocation = INVEST * initial_weights / initial_price
    initial_allocation[np.isinf(initial_allocation)] = 0
    
    initial_positions = initial_allocation * initial_price
        
    current_allocation = initial_allocation
    
    day_count = 1
    fund_returns[fund['name']] = {}
    for i, current_price in coin_prices.iterrows():
        day_count += 1

        current_positions = current_allocation * current_price
        fund_returns[fund['name']][i] = current_positions

        # rebalance
        current_portfolio_value = current_positions.sum()
        if fund['rebalance'] > 0 and day_count > fund['rebalance'] and current_price.min() > 0:
            #update the allocation for each token (preform the rebalance)
            current_allocation = current_portfolio_value * initial_weights / current_price
            #restart counting for next rebalance
            day_count = 1
        #increment time


In [263]:
pd.DataFrame(fund_returns['ETH40, BTC40, USD20 @ 7']).T.head(15)

,eth_price,btc_price,usd_price
2017-01-01,400.000000,400.000000,200.000000
2017-01-02,397.522167,416.071053,200.000000
2017-01-03,428.082109,418.954714,200.000000
2017-01-04,507.227013,442.847903,200.000000
2017-01-05,509.121827,430.489357,200.000000
2017-01-06,486.189724,382.284436,200.000000
2017-01-07,474.966598,356.497918,200.000000
2017-01-08,421.406432,433.275455,206.292903
2017-01-09,439.933965,424.667005,206.292903
2017-01-10,438.794459,428.653228,206.292903


In [264]:
fund_computed_plot = []
for fund in fund_returns:
    print(fund)
    fund_df = pd.DataFrame(fund_returns[fund]).T
    
    fund_value = fund_df.sum(axis=1)
    fund_performance = (fund_value.iloc[-1] / fund_value.iloc[0]) * 100
    fund_performance = round(fund_performance, 3)
    fund_plot = go.Scatter(
        x = fund_value.index,
        y = fund_value.tolist(),
        mode = 'lines',
        name = '{0} -> {1}%'.format(fund, fund_performance))
#         name = fund)
    
#     fund_computed[fund] = {}
#     fund_computed[fund]['value'] = fund_value
#     fund_computed[fund]['performance'] = fund_performance
#     fund_computed[fund]['plot'] = fund_plot
    
    fund_computed_plot.append(fund_plot)

ETH40, BTC40, USD20 @ 7
ETH25, BTC25, USD20 @ 30
ETH50, BTC50 @ 15
ETH50, BTC50 @ HODL


In [267]:
layout = go.Layout(
    title = "Portfolio value over time",
    autosize=True,
    showlegend=True,
    legend=dict(x=0.7, y=0.9))

fig = dict(data=fund_computed_plot, layout = layout)
py.iplot(fig)

PlotlyRequestError: Account limit reached: Your account is limited to creating 25 charts. To continue, you can override or delete existing charts or you can upgrade your account at: https://plot.ly/products/cloud